**Due Date: Monday, October 19th, 11:59pm**

- Fill out the missing parts.
- Answer the questions (if any) in a separate document or by adding a new `Text` block inside the Colab.
- Save the notebook by going to the menu and clicking `File` > `Download .ipynb`.
- Make sure the saved version is showing your solutions.
- Send the saved notebook by email to your TA.

In [1]:
import numpy as np

np.random.seed(0)

Simulate a dataset of 100 coin flips for a coin with $p$ = P(head) = 0.6.

In [2]:
n = 100   # number of coin flips
p = 0.6   # probability of getting a head (not a fair coin)

# A coin toss experiment can be modeled with a binomial distribution
# if we set n=1 (one trial), which is equivalent to a Bernoulli distribution
y = np.random.binomial(n=1, p=p, size=n)
y

array([1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1])

## Point Estimation

Estimate the value of $p$ using the data.

In [3]:
def estimator(y):
  return np.mean(y)

p_hat = estimator(y)
p_hat

0.62

## Bootstrap

Estimate the standard error of $\hat{p}$ using bootstrap.

In [4]:
def bootstrap_se_est(y, stat_function, B=1000):
  # 1. Generate bootstrap samples from the observed/simulated data (i.e. y)
  # 2. Compute the statistic (using stat_function passed) on the bootstrap 
  #    samples
  # 3. Compute the standard error -> std dev
  t_boot_list = [stat_function(np.random.choice(y, len(y), replace=True)) 
                 for _ in range(B)]
  return np.std(t_boot_list)

print("Standard error of p_hat computed by bootstrap:")
print(bootstrap_se_est(y, estimator))

Standard error of p_hat computed by bootstrap:
0.04889048066853097


Validate the estimated standard error by computing it analytically.

In [5]:
def estimator_analytical_se(p, n):
  return np.sqrt(p * (1-p) / n)

print("Analytical standard error for the estimator: ", estimator_analytical_se(p, n))

Analytical standard error for the estimator:  0.04898979485566356


Estimate the 95% confidence interval for $p$.

In [6]:
def confidence_interval_95_for_p(y):
  ci_lower = estimator(y) - 1.96*bootstrap_se_est(y, estimator)
  ci_higher = estimator(y) + 1.96*bootstrap_se_est(y, estimator)
  return (ci_lower, ci_higher)

lower, higher = confidence_interval_95_for_p(y)
print("95% confidence interval for p: ({},{})".format(lower, higher))

95% confidence interval for p: (0.5254445619916019,0.717033857596202)


Validate the 95% confidence interval for $p$.

In [7]:
ci_contains_p_flags = []
for sim in range(1000):
  y = np.random.binomial(n=1, p=p, size=n)
  ci_lower, ci_higher = confidence_interval_95_for_p(y)
  if ci_lower < p and p < ci_higher:
    ci_contains_p_flags.append(1)
  else: 
    ci_contains_p_flags.append(0)

coverage = np.mean(ci_contains_p_flags)
print("Coverage of 95% confidence interval for p: ", coverage)

Coverage of 95% confidence interval for p:  0.93


## Bayesian Inference

**[Optional]**

Estimate $p$ using Bayesian inference. As the prior for $p$ use Normal(0.5, 0.1).

In [8]:
!pip install pystan

In [9]:
import pystan

In [10]:
model_code = """
data {
  int<lower=0> n;
  int<lower=0,upper=1> y[n];
}
parameters {
  real<lower=0,upper=1> p;
}
model {
  p ~ normal(0.5, 0.1);
  for (i in 1:n)
    y[i] ~ bernoulli(p);
}
"""

model = pystan.StanModel(model_code=model_code)
fit = model.sampling(data={"n": n, "y": y}, iter=2000, chains=4, n_jobs=4)
print(fit.stansummary())

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_da99211d8d54f3aa1eb8363291e9586e NOW.


CompileError: command 'C:\\Program Files (x86)\\Microsoft Visual Studio\\2019\\Community\\VC\\Tools\\MSVC\\14.26.28801\\bin\\HostX86\\x64\\cl.exe' failed with exit status 2

Compute the Bayesian inference results if our data contains 20 coin tosses instead.

In [ ]:
n = 20
p = 0.6
y = np.random.binomial(1, p, n)

model = pystan.StanModel(model_code=model_code)
fit = model.sampling(data={"n": n, "y": y}, iter=2000, chains=4, n_jobs=4)
print(fit.stansummary())